In [227]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors,cm,colorbar
import scipy
import shapefile
import re
import json

In [228]:
#https://www.kaggle.com/benhamner/2016-us-election

In [229]:
61201031 +62523126 +6464094 

130188251

## THIS copy is for the map app

## Load election results

In [230]:
el=pd.read_csv("2016_US_County_Level_Presidential_Results.csv")

In [231]:
by_state=el[~((el.state_abbr=='AK')& (el.combined_fips!=2013))].groupby('state_abbr')['votes_dem','votes_gop','total_votes'].aggregate(sum)

In [232]:
el['per_dem']=100*el['per_dem']
el['per_gop']=100*el['per_gop']

In [233]:
by_state['per_dem']=100*by_state['votes_dem']/by_state['total_votes']
by_state['per_gop']=100*by_state['votes_gop']/by_state['total_votes']

In [234]:
oglala=el.index[(el['state_abbr']=='SD')& (el['county_name']=='Oglala County')].values[0]
el.loc[oglala,'combined_fips'] = 46102
el.loc[oglala,'county_name'] = 'Oglala Lakota County'

In [235]:
kusilvak=el.index[(el['state_abbr']=='AK')& (el['combined_fips']==2270)].values[0]
el.loc[kusilvak,'combined_fips'] = 2158
el.loc[kusilvak,'county_name'] = 'Kusilvak'

## Shapefile for counties

In [236]:
sf=shapefile.Reader("cb_2015_us_county_5m.shp")
shapes=sf.shapes()
col_names=[x[0] for x in sf.fields][1:]
records=pd.DataFrame([dict(zip(*[col_names, x])) for x in sf.records()])

In [237]:
outlying=[2,60,3,81,7,64,14,66,84,86,67,89,71,76,69,70,95,72,74,52,78,79,15]

In [238]:
records['combined_fips']=[int(x.STATEFP+x.COUNTYFP) for i,x in records.iterrows()]
records['points']=[x.points for x in shapes]
records['parts']=[x.parts for x in shapes]
records['geo_interface']=[x.__geo_interface__ for x in shapes]

In [239]:
merged=pd.merge(el[['combined_fips','per_dem','per_gop','total_votes']],
         records,on='combined_fips',how='inner'
)

## States

In [240]:
state_shapes_file=shapefile.Reader("cb_2015_us_state_5m.shp")
state_shapes=state_shapes_file.shapes()
state_col_names=[x[0] for x in state_shapes_file.fields][1:]
state_records=pd.DataFrame([dict(zip(*[state_col_names, x])) for x in state_shapes_file.records()])
state_records['points']=[x.points for x in state_shapes]
state_records['parts']=[x.parts for x in state_shapes]
state_records['geo_interface']=[x.__geo_interface__ for x in state_shapes]
state_records=state_records.set_index('STUSPS')
merged_states=pd.merge(state_records,by_state,left_index=True,right_index=True)

## Load Country facts

In [241]:
import sqlite3

In [242]:
conn=sqlite3.connect('database.sqlite')
curs=conn.cursor()
res=curs.execute("select name from sqlite_master where type='table'")
res.fetchall()

[(u'primary_results',), (u'county_facts',), (u'county_facts_dictionary',)]

In [243]:
res=curs.execute("select * from county_facts")
country_facts=pd.DataFrame(res.fetchall())
res=curs.execute("select * from county_facts_dictionary")
header=pd.DataFrame(res.fetchall())

In [244]:
lst=['combined_fips','name','abbr']
lst.extend(header[0].values)
country_facts.columns=lst

In [245]:
header=header.set_index(0)
header[0]=header.index

In [246]:
header_percent=header[~header[1].apply(lambda x: re.search("percent|rate|pct",x) is None)]
header_extensive=header[header[1].apply(lambda x: re.search("percent|rate|minutes|median|Median|per |Per |pct",x) is None)]
header_extensive=header_extensive.drop('PST045214')

In [247]:

for i,x in header.iterrows():
    country_facts[x[0]]=country_facts[x[0]].astype(float)
#for i,x in header_percent.iterrows():
#    country_facts[x[0]]=country_facts[x[0]]

for i,x in header_extensive.iterrows():
    country_facts[x[0]]=country_facts[x[0]]/country_facts['PST045214']

In [248]:
country_facts['HSG495213']=country_facts['HSG495213']/1000

In [249]:
country_facts["INC910213"]=country_facts["INC910213"]/1000

In [250]:
country_facts["INC110213"]=country_facts["INC110213"]/1000 

In [251]:
country_facts["BPS030214"]=country_facts["BPS030214"]*1000 

### about states again

In [252]:
#header['percent_zeros']=[np.mean(merged3[x]==0) for x in header[0]]
merged_states['abbr']=merged_states.index

In [253]:
tmp=country_facts[country_facts.abbr=='']
del tmp['abbr']
merged3_states=pd.merge(merged_states,tmp,left_on='NAME',right_on='name')

In [254]:
alaska=np.array(np.array(state_shapes_file.shapes())[np.array([x[4]=='AK' for x in state_shapes_file.records()])][0].points)
texas=merged3_states[merged3_states.NAME=='Texas'].points
al_center=np.array(alaska).mean(axis=0)
tex_center=np.array(texas[42]).mean(axis=0)
small_al=0.5*(np.array(alaska)-al_center)+al_center
alaska_moved=np.array([small_al[:,0],small_al[:,1]-al_center[1]+tex_center[1]])

In [255]:
hawaii=merged3_states[merged3_states.NAME=='Hawaii'].points
hi_center=np.array(hawaii[10]).mean(axis=0)

In [256]:
olala=country_facts[(country_facts['abbr']=='SD')& (country_facts['name']=='Shannon County')].index[0]
country_facts.loc[olala,'combined_fips'] = 46102
country_facts.loc[olala,'name'] = 'Oglala Lakota County'

In [257]:
kusilvak=country_facts[(country_facts['abbr']=='AK')& (country_facts['name']=='Wade Hampton Census Area')].index[0]
country_facts.loc[kusilvak,'combined_fips'] = 2158
country_facts.loc[kusilvak,'name'] = 'Kusilvak'

In [258]:
#row=merged[['combined_fips','STATEFP','per_dem','per_gop','total_votes']].loc[0]

In [259]:
#to_append=pd.DataFrame([{'combined_fips':x,'STATEFP':'02','per_dem':row['per_dem'],'per_gop':row['per_gop'],'total_votes':'NA'} for x in country_facts[country_facts.abbr=='AK'].combined_fips])

In [260]:
tab1=merged[['combined_fips','STATEFP','per_dem','per_gop','total_votes']]
#to_append.index=pd.RangeIndex(start=tab1.shape[0], stop=tab1.shape[0]+29, step=1)
#tab1=pd.concat([tab1,to_append])

In [261]:
merged2=pd.merge(tab1,country_facts,on='combined_fips')
merged3=pd.merge(merged2,records,on='combined_fips')
merged3['STATEFP']=merged3.STATEFP_x
#merged3.drop('PST045214',axis=1,inplace=True)
#header.drop('PST045214',inplace=True)

In [262]:
from IPython.display import HTML
HTML(header.T.to_html())

,PST045214,PST040210,PST120214,POP010210,AGE135214,AGE295214,AGE775214,SEX255214,RHI125214,RHI225214,RHI325214,RHI425214,RHI525214,RHI625214,RHI725214,RHI825214,POP715213,POP645213,POP815213,EDU635213,EDU685213,VET605213,LFE305213,HSG010214,HSG445213,HSG096213,HSG495213,HSD410213,HSD310213,INC910213,INC110213,PVY020213,BZA010213,BZA110213,BZA115213,NES010213,SBO001207,SBO315207,SBO115207,SBO215207,SBO515207,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
1,"Population, 2014 estimate","Population, 2010 (April 1) estimates base","Population, percent change - April 1, 2010 to ...","Population, 2010","Persons under 5 years, percent, 2014","Persons under 18 years, percent, 2014","Persons 65 years and over, percent, 2014","Female persons, percent, 2014","White alone, percent, 2014","Black or African American alone, percent, 2014","American Indian and Alaska Native alone, perce...","Asian alone, percent, 2014",Native Hawaiian and Other Pacific Islander alo...,"Two or More Races, percent, 2014","Hispanic or Latino, percent, 2014","White alone, not Hispanic or Latino, percent, ...","Living in same house 1 year & over, percent, 2...","Foreign born persons, percent, 2009-2013","Language other than English spoken at home, pc...","High school graduate or higher, percent of per...","Bachelor's degree or higher, percent of person...","Veterans, 2009-2013","Mean travel time to work (minutes), workers ag...","Housing units, 2014","Homeownership rate, 2009-2013","Housing units in multi-unit structures, percen...","Median value of owner-occupied housing units, ...","Households, 2009-2013","Persons per household, 2009-2013",Per capita money income in past 12 months (201...,"Median household income, 2009-2013","Persons below poverty level, percent, 2009-2013","Private nonfarm establishments, 2013","Private nonfarm employment, 2013","Private nonfarm employment, percent change, 20...","Nonemployer establishments, 2013","Total number of firms, 2007","Black-owned firms, percent, 2007",American Indian- and Alaska Native-owned firms...,"Asian-owned firms, percent, 2007",Native Hawaiian- and Other Pacific Islander-ow...,"Hispanic-owned firms, percent, 2007","Women-owned firms, percent, 2007","Manufacturers shipments, 2007 ($1,000)","Merchant wholesaler sales, 2007 ($1,000)","Retail sales, 2007 ($1,000)","Retail sales per capita, 2007","Accommodation and food services sales, 2007 ($...","Building permits, 2014","Land area in square miles, 2010","Population per square mile, 2010"
0,PST045214,PST040210,PST120214,POP010210,AGE135214,AGE295214,AGE775214,SEX255214,RHI125214,RHI225214,RHI325214,RHI425214,RHI525214,RHI625214,RHI725214,RHI825214,POP715213,POP645213,POP815213,EDU635213,EDU685213,VET605213,LFE305213,HSG010214,HSG445213,HSG096213,HSG495213,HSD410213,HSD310213,INC910213,INC110213,PVY020213,BZA010213,BZA110213,BZA115213,NES010213,SBO001207,SBO315207,SBO115207,SBO215207,SBO515207,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210


## Write down maxmins

In [263]:
lst=["per_dem","per_gop","total_votes",u'PST120214',     u'AGE135214',     u'AGE295214',
           u'AGE775214',     u'SEX255214',     u'RHI225214',     u'RHI425214',
           u'RHI625214',     u'RHI725214',     u'RHI825214',     u'POP715213',
           u'POP645213',     u'POP815213',     u'EDU635213',     u'EDU685213',
           u'VET605213',     u'LFE305213',     u'HSG010214',     u'HSG445213',
           u'HSG096213',     u'HSG495213',     u'HSD310213',     u'INC910213',
           u'INC110213',     u'PVY020213',     u'BZA010213',     u'BZA110213',
           u'BZA115213',     u'NES010213',     u'SBO001207',     u'SBO315207',
           u'SBO415207',     u'SBO015207',     u'MAN450207',     u'WTN220207',
           u'RTN131207',     u'AFN120207',     u'BPS030214',     u'POP060210']

hd1=header.append(pd.DataFrame({"per_dem":pd.Series({1:"Votes for Democrats, percent, 2016",0:"per_dem"}),
                                "per_gop":pd.Series({1:"Votes for GOP, percent, 2016",0:"per_gop"}),
                                "total_votes":pd.Series({1:"Total votes",0:"total_votes"})
                               }).T)


In [264]:
hd1[1]=np.array([u'Population, 2014 estimate',u'Population, 2010 (April 1) estimates base',
       u'Population, percent change - April 1, 2010 to July 1, 2014',
       u'Population, 2010', u'Persons under 5 years, percent, 2014',
       u'Persons under 18 years, percent, 2014',
       u'Persons 65 years and over, percent, 2014',
       u'Female persons, percent, 2014', u'White alone, percent, 2014',
       u'Black or African American alone, percent, 2014',
       u'American Indian and Alaska Native alone, percent, 2014',
       u'Asian alone, percent, 2014',
       u'Native Hawaiian and Other Pacific Islander alone, percent, 2014',
       u'Two or More Races, percent, 2014',
       u'Hispanic or Latino, percent, 2014',
       u'White alone, not Hispanic or Latino, percent, 2014',
       u'Living in same house 1 year & over, percent, 2009-2013',
       u'Foreign born persons, percent, 2009-2013',
       u'Language other than English spoken at home, pct age 5+, 2009-2013',
       u'High school graduate or higher, percent of persons age 25+, 2009-2013',
       u"Bachelor's degree or higher, percent of persons age 25+, 2009-2013",
       u'Veterans, percent, 2009-2013',
       u'Mean travel time to work (minutes), workers age 16+, 2009-2013',
       u'Housing units per capita, 2014', u'Homeownership rate, 2009-2013',
       u'Housing units in multi-unit structures, percent, 2009-2013',
       u'Median value of owner-occupied housing units, 2009-2013 ($1,000)',
       u'Households, 2009-2013', u'Persons per household, 2009-2013',
       u'Per capita money income in past 12 months, 2009-2013 ($1,000, 2013 dollars)',
       u'Median household income, 2009-2013 ($1,000)',
       u'Persons below poverty level, percent, 2009-2013',
       u'Private nonfarm establishments per capita, 2013',
       u'Private nonfarm employment per capita,  2013',
       u'Private nonfarm employment, percent change, 2012-2013',
       u'Nonemployer establishments per capita, 2013', u'Total number of firms per capita, 2007',
       u'Black-owned firms, percent, 2007',
       u'American Indian- and Alaska Native-owned firms, percent, 2007',
       u'Asian-owned firms, percent, 2007',
       u'Native Hawaiian- and Other Pacific Islander-owned firms, percent, 2007',
       u'Hispanic-owned firms, percent, 2007',
       u'Women-owned firms, percent, 2007',
       u'Manufacturers shipments per capita, 2007 ($1,000)',
       u'Merchant wholesaler sales per capita, 2007 ($1,000)',
       u'Retail sales per capita, 2007 ($1,000)', u'Retail sales per capita, 2007',
       u'Accommodation and food services sales per capita, 2007 ($1,000)',
       u'Building permits per 1000 people, 2014', u'Land area in square miles, 2010',
       u'Population per square mile, 2010',
       'Votes for Democrats, percent, 2016',
       'Votes for GOP, percent, 2016', 'Total votes'], dtype=object)

In [265]:
lst=list(hd1.index)

In [266]:
maxmins_counties = {"mins":merged3[lst].quantile(0.1).to_dict(),"maxs":merged3[lst].quantile(0.9).to_dict(),
                "names":hd1.T[lst].loc[1].to_dict()}
maxmins_states = {"mins":merged3_states[lst].quantile(0.1).to_dict(),"maxs":merged3_states[lst].quantile(0.9).to_dict(),
                "names":hd1.T[lst].loc[1].to_dict()}

In [267]:
maxmins_counties['maxs']['SBO515207']=merged3['SBO515207'].quantile(0.99)

In [268]:
jsn=json.dumps({"counties":maxmins_counties,"states":maxmins_states})
file_json=open("maxmins.json","wt")
file_json.write(jsn)
file_json.close()

## Convert to geoJSON

In [269]:
def ff(row):
# if (len(row.parts)<2):
   #     tp = 'Polygon'
#     pts = [row.points]
   # else:
#     tp = 'MultiPolygon'
   #     pts=[[row.points[row.parts[i]:row.parts[i+1]] for i in range(len(row.parts)-1)]]
    dt={'geometry':row.geo_interface,"type":'Feature',"properties":dict(row)}
    del dt['properties']['points']
    del dt['properties']['parts']
    del dt['properties']['geo_interface']
    return dt

In [270]:
features=[ff(row) for i,row in merged3.iterrows()]

In [271]:
features_states=[ff(row) for i,row in merged3_states.iterrows()]

In [272]:
shrink=0.2
def trans1(x):
    y=[[[[(p[0]-al_center[0])*shrink+al_center[0]+23,(p[1]-al_center[1])*shrink+tex_center[1]-2] for p in z] for z in w] for w in x]
    #print('trans1', x,y)
    return y
def trans2(x):
    y=[[[(p[0]-al_center[0])*shrink+al_center[0]+23,(p[1]-al_center[1])*shrink+tex_center[1]-2] for p in w] for w in x]
    #print('trans2', x,y)
    return y

In [273]:
features_states[0]['geometry']['coordinates']=trans1(features_states[0]['geometry']['coordinates'])

In [274]:
ak_ind=np.arange(len(features))[np.array([x['properties']['abbr']=='AK' for x in features])]
for i in ak_ind:
    if (features[i]['geometry']['type']=='MultiPolygon'):
        features[i]['geometry']['coordinates']=trans1(features[i]['geometry']['coordinates']);
    else: 
        features[i]['geometry']['coordinates']=trans2(features[i]['geometry']['coordinates']);

In [275]:
shrink=1
def trans3(x):
    y=[[[[(p[0]-hi_center[0])*shrink+al_center[0]+24,p[1]+5] for p in z] for z in w] for w in x]
    #print('trans1', x,y)
    return y
def trans4(x):
    y=[[[(p[0]-hi_center[0])*shrink+al_center[0]+24,p[1]+5] for p in w] for w in x]
    #print('trans2', x,y)
    return y
features_states[10]['geometry']['coordinates']=trans3(features_states[10]['geometry']['coordinates'])
hi_ind=np.arange(len(features))[np.array([x['properties']['abbr']=='HI' for x in features])]
for i in hi_ind:
    if (features[i]['geometry']['type']=='MultiPolygon'):
        features[i]['geometry']['coordinates']=trans3(features[i]['geometry']['coordinates']);
    else: 
        features[i]['geometry']['coordinates']=trans4(features[i]['geometry']['coordinates']);

In [276]:
final_json={"type":"FeatureCollection","features":features}
json_output=open("test.json","wt")
json.dump(final_json,json_output)
json_output.close()

In [277]:
final_json={"type":"FeatureCollection","features":features_states}
json_output=open("test_states.json","wt")
json.dump(final_json,json_output)
json_output.close()

In [285]:
[-148.716968,61.288254],[-157.524452,21.146768]

32.033010540915981

In [287]:
shrink=0.2
[(-148.716968-al_center[0])*shrink+al_center[0]+23,(61.288254-al_center[1])*shrink+tex_center[1]-2]

[-121.30642032803243, 29.679606821701242]

In [288]:
shrink=1
[(-157.524452-hi_center[0])*shrink+al_center[0]+24,21.146768+5]

[-119.67543714272617, 26.146768]

In [280]:
al_center[1]

58.346499350981581

In [281]:
tex_center[1]

31.091255891897557